In [27]:
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

# Load all JSON files from the tournaments_data folder
data_folder = "tournaments_data"
all_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.json')]

# Initialize lists to store extracted data
tournament_info_list = []
team_members_list = []
match_cards_list = []
maps_list = []

# Helper function to extract map data
def extract_map_data(match_id, match_data):
    maps = []
    for key, value in match_data.items():
        if key.startswith('map'):
            map_num = key[3:]
            map_info = value if isinstance(value, dict) else {}
            map_info['map_num'] = map_num
            map_info['match_card_id'] = match_id
            maps.append(map_info)
    return maps

# ID counters
tournament_id_counter = 1
team_id_counter = 1
match_id_counter = 1

# Process each file with a progress bar
for file in tqdm(all_files, desc="Processing JSON files"):
    with open(file, 'r') as f:
        data = json.load(f)
        
        # Extract tournament_info
        tournament_info = data.get("tournament_info", [])
        if tournament_info:
            tournament_info = tournament_info[0]
        else:
            tournament_info = {}
        
        tournament_info["tournament_id"] = tournament_id_counter
        tournament_info_list.append(tournament_info)
        
        # Extract team members
        team_cards = data.get("team_cards", [])
        for team in team_cards:
            if not team.get("team"):
                continue
            team_id = team_id_counter
            team_id_counter += 1
            members = team.get("members", [])
            for member in members:
                if not member.get("name"):
                    continue
                team_member_info = {
                    "team": team.get("team", ""),
                    "tournament_id": tournament_id_counter,
                    "team_id": team_id
                }
                team_member_info.update(member)  # Add all keys from member
                team_members_list.append(team_member_info)
        
        # Extract match cards
        match_cards = data.get("match_cards", [])
        for match in match_cards:
            if not match.get("winner"):
                continue
            match_id = match_id_counter
            match_id_counter += 1
            match['tournament_id'] = tournament_id_counter
            match['match_card_id'] = match_id
            match_cards_list.append(match)
            
            # Extract maps data from match cards
            maps = extract_map_data(match_id, match)
            maps_list.extend(maps)
    
    tournament_id_counter += 1

# Convert lists to DataFrames
tournament_info_df = pd.DataFrame(tournament_info_list)
team_members_df = pd.DataFrame(team_members_list)
match_cards_df = pd.DataFrame(match_cards_list)
maps_df = pd.DataFrame(maps_list)

Processing JSON files: 100%|██████████| 3685/3685 [00:00<00:00, 4276.49it/s]


In [28]:
# Convert all empty whitespace values to NaN
def convert_whitespace_to_nan(df):
    return df.applymap(lambda x: np.nan if isinstance(x, str) and x.strip() == '' else x)

tournament_info_df = convert_whitespace_to_nan(tournament_info_df)
team_members_df = convert_whitespace_to_nan(team_members_df)
match_cards_df = convert_whitespace_to_nan(match_cards_df)
maps_df = convert_whitespace_to_nan(maps_df)

# Function to validate date
def is_valid_date(date_str):
    try:
        pd.to_datetime(date_str, format="%Y-%m-%d", errors='raise')
        return True
    except ValueError:
        return False

# Function to validate date and convert to datetime
def to_datetime(date_str):
    try:
        return pd.to_datetime(date_str, format="%Y-%m-%d", errors='raise')
    except ValueError:
        return pd.NaT
    
# Function to compare dates and return the least recent valid date
def get_least_recent_date(dates):
    valid_dates = [date for date in dates if pd.notna(to_datetime(date))]
    return min(valid_dates) if valid_dates else None

# Calculate agg_date
tournament_info_df['agg_date'] = tournament_info_df.apply(lambda row: get_least_recent_date(
    [row.get('date', ''), row.get('sdate', ''), row.get('edate', '')]
), axis=1)

# Convert agg_date to just the date part (YYYY-MM-DD)
#tournament_info_df['agg_date'] = pd.to_datetime(tournament_info_df['agg_date']).dt.date

# Drop rows with no valid date
tournament_info_df = tournament_info_df[tournament_info_df['agg_date'].notna()]

# Sort by agg_date and create the date_order column
tournament_info_df = tournament_info_df.sort_values(by='agg_date').reset_index(drop=True)
tournament_info_df['date_order'] = tournament_info_df.index + 1

# Drop rows where liquipediatier column is 5.0
tournament_info_df = tournament_info_df[tournament_info_df['liquipediatier'] != 5.0]

# Keep only specified columns
columns_to_keep = ['name', 'series', 'organizer', 'type', 'city', 'country', 'prizepool', 'prizepoolusd', 'format', 'date', 'sdate', 'edate', 'liquipediatier', 'team_number', 'agg_date', 'tournament_id', 'date_order']
tournament_info_df = tournament_info_df[columns_to_keep]

# Process match_cards_df
# Fill missing date values from tournament agg_date
match_cards_df = match_cards_df.merge(tournament_info_df[['tournament_id', 'agg_date']], on='tournament_id', how='left', suffixes=('', '_tournament'))
match_cards_df['date'] = match_cards_df.apply(lambda row: row['date'] if not pd.isna(row['date']) else row['agg_date'], axis=1)
match_cards_df.drop(columns=['agg_date'], inplace=True)

# Drop rows without opponent1 or opponent2
match_cards_df = match_cards_df.dropna(subset=['opponent1', 'opponent2'])

# Drop rows where opponent1 or opponent2 is 'BYE'
match_cards_df = match_cards_df[
    (match_cards_df['opponent1'] != 'BYE') & (match_cards_df['opponent2'] != 'BYE')
]

# Keep only specified columns
columns_to_keep = ['date', 'date_timezone', 'opponent1', 'opponent2', 'opponent1_score', 'opponent2_score', 'winner', 'format', 'date_time', 'key', 'tournament_id', 'match_card_id']
match_cards_df = match_cards_df[columns_to_keep]

# Keep only specified columns
columns_to_keep = ['map', 'mode', 'score1', 'score2', 'winner', 'map_num', 'match_card_id'] 
maps_df = maps_df[columns_to_keep]

# Drop related entries in other dataframes for removed tournaments
valid_tournament_ids = set(tournament_info_df['tournament_id'])

# Drop match_cards_df records where tournament_id is not in tournament_id in tournament_info_df
match_cards_df = match_cards_df[match_cards_df['tournament_id'].isin(valid_tournament_ids)]

# Drop maps where match_card_id is not in match_cards_df.match_card_id
valid_match_card_ids = set(match_cards_df['match_card_id'])
maps_df = maps_df[maps_df['match_card_id'].isin(valid_match_card_ids)]

# Drop teams where tournament_id is not in tournament_info_df
team_members_df = team_members_df[team_members_df['tournament_id'].isin(valid_tournament_ids)]

# Track if each tournament has players or not for later.
tournament_info_df['has_players'] = tournament_info_df['tournament_id'].isin(team_members_df['tournament_id'])


C:\Users\jorda\AppData\Local\Temp\ipykernel_35436\1363133916.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: np.nan if isinstance(x, str) and x.strip() == '' else x)


In [29]:
import pandas as pd
from tqdm import tqdm
from difflib import get_close_matches
import re
import math
import json

# Set up tqdm with pandas
tqdm.pandas()

# Load the acronym map
with open('acronym_map.json', 'r') as file:
    acronym_map = json.load(file)

# Map opponent acronyms to full names, retain original name if not found
def map_acronyms(team_name):
    return acronym_map.get(team_name, team_name)

match_cards_df['opponent1'] = match_cards_df['opponent1'].apply(map_acronyms)
match_cards_df['opponent2'] = match_cards_df['opponent2'].apply(map_acronyms)

# Terms to remove
terms_to_remove = ['GAMING', 'TEAM', 'ESPORTS', 'ESPORT', 'CLUB']

def clean_team_name(name):
    # Remove specified terms
    for term in terms_to_remove:
        name = name.replace(term, '')
    # Remove non-alphanumeric characters
    name = re.sub(r'\W+', '', name)
    return name.strip().upper()  # Convert to upper case for uniformity

# Create clean_team column in team_members_df
team_members_df['clean_team'] = team_members_df['team'].apply(clean_team_name)
tournament_id_to_date_order = tournament_info_df.set_index('tournament_id')['date_order'].to_dict()
tournament_id_to_liquipediatier = tournament_info_df.set_index('tournament_id')['liquipediatier'].to_dict()
tournament_id_to_agg_date = tournament_info_df.set_index('tournament_id')['agg_date'].to_dict()

# Add date_order, liquipediatier, and agg_date to match_cards_df using the mapping
match_cards_df['date_order'] = match_cards_df['tournament_id'].map(tournament_id_to_date_order)
match_cards_df['liquipediatier'] = match_cards_df['tournament_id'].map(tournament_id_to_liquipediatier)
match_cards_df['agg_date'] = match_cards_df['tournament_id'].map(tournament_id_to_agg_date)

# Create unique_teams dataframe directly from team_members_df
unique_teams = team_members_df[['clean_team', 'team_id', 'tournament_id']].drop_duplicates()

# Add date_order, liquipediatier, and agg_date to unique_teams using the mapping
unique_teams['date_order'] = unique_teams['tournament_id'].map(tournament_id_to_date_order)
unique_teams['liquipediatier'] = unique_teams['tournament_id'].map(tournament_id_to_liquipediatier)
unique_teams['agg_date'] = unique_teams['tournament_id'].map(tournament_id_to_agg_date)

# Initial matching within the same tournament
def find_closest_team_id_within_tournament(row, unique_teams, threshold=0.8):
    tournament_teams = unique_teams[unique_teams['tournament_id'] == row['tournament_id']]
    team_names = tournament_teams['clean_team'].tolist()
    team_ids = tournament_teams['team_id'].tolist()
    
    best_team1_id, best_team2_id = None, None
    
    if team_names:
        matches1 = get_close_matches(row['clean_opponent1'], team_names, n=1, cutoff=threshold)
        matches2 = get_close_matches(row['clean_opponent2'], team_names, n=1, cutoff=threshold)
        
        if matches1:
            best_team1_id = team_ids[team_names.index(matches1[0])]
        if matches2:
            best_team2_id = team_ids[team_names.index(matches2[0])]
    
    return best_team1_id, best_team2_id

# Apply initial matching with progress bar
match_cards_df['clean_opponent1'] = match_cards_df['opponent1'].apply(clean_team_name)
match_cards_df['clean_opponent2'] = match_cards_df['opponent2'].apply(clean_team_name)

# Initial matching
team_ids = match_cards_df.progress_apply(lambda row: find_closest_team_id_within_tournament(row, unique_teams, 0.8), axis=1)
match_cards_df[['team1_id', 'team2_id']] = pd.DataFrame(team_ids.tolist(), index=match_cards_df.index)

# Function to find the closest matching team_id with index distance, liquipediatier constraint, and identical agg_date check
def find_closest_team_id_with_index_distance(row, unique_teams, threshold=0.8, max_index=1):
    date_order = row['date_order']
    agg_date = row['agg_date']
    tier = row['liquipediatier']
    date_range = range(date_order - max_index, date_order + 1)  # Looking backward in time
    
    candidate_teams = unique_teams[((unique_teams['date_order'].isin(date_range)) | 
                                   (unique_teams['agg_date'] == agg_date)) & 
                                   (unique_teams['liquipediatier'].between(tier - 1, tier + 1))]
    team_names = candidate_teams['clean_team'].tolist()
    team_ids = candidate_teams['team_id'].tolist()
    
    best_team1_id, best_team2_id = None, None
    
    if team_names:
        matches1 = get_close_matches(row['clean_opponent1'], team_names, n=1, cutoff=threshold)
        matches2 = get_close_matches(row['clean_opponent2'], team_names, n=1, cutoff=threshold)
        
        if matches1:
            best_team1_id = team_ids[team_names.index(matches1[0])]
        if matches2:
            best_team2_id = team_ids[team_names.index(matches2[0])]
    
    return best_team1_id, best_team2_id

# Loop to progressively reduce threshold and increase index distance
def progressively_match_teams(match_cards_df, unique_teams):
    threshold = 1
    max_index = 1

    while True:
        missing_team1_ids = match_cards_df['team1_id'].isna()
        missing_team2_ids = match_cards_df['team2_id'].isna()

        if not missing_team1_ids.any() and not missing_team2_ids.any():
            break

        print(f"Threshold {threshold}. Max Index {max_index}")

        team_ids = match_cards_df[missing_team1_ids | missing_team2_ids].progress_apply(
            lambda row: find_closest_team_id_with_index_distance(row, unique_teams, threshold, max_index),
            axis=1
        )
        
        match_cards_df.loc[missing_team1_ids, 'team1_id'] = team_ids.apply(lambda x: x[0])
        match_cards_df.loc[missing_team2_ids, 'team2_id'] = team_ids.apply(lambda x: x[1])

        # Decrease threshold and increase index distance
        threshold -= 0.02
        max_index = math.ceil(max_index * 1.5)

        # Stop if threshold is too low
        if threshold < 0.7:
            break

# Apply the progressive matching
progressively_match_teams(match_cards_df, unique_teams)


100%|██████████| 16677/16677 [00:02<00:00, 5579.88it/s]


Threshold 1. Max Index 1


 36%|███▌      | 3027/8516 [00:01<00:03, 1575.34it/s]

In [ ]:
from collections import defaultdict

# Display the initial info of the DataFrame
team_members_df.info()

# Helper function to find the most common value in a list
def most_common(lst):
    return max(set(lst), key=lst.count)

# Helper function to create lookup dictionary
def create_lookup(df, key_cols, value_col):
    lookup = defaultdict(list)
    for _, row in df[df[value_col].notna()].iterrows():
        key = tuple(row[col] for col in key_cols)
        lookup[key].append(row[value_col])
    return {
        key: most_common(values)
        for key, values in lookup.items()
    }

# Impute missing values based on lookup dictionary
def impute_missing_values(df, lookup, key_cols, value_col):
    for idx, row in df[df[value_col].isna()].iterrows():
        key = tuple(row[col] for col in key_cols)
        if key in lookup:
            df.at[idx, value_col] = lookup[key]

# Step 2: Impute missing position values
position_lookup = create_lookup(team_members_df, ["name", "flag", "role_type"], "position")
impute_missing_values(team_members_df, position_lookup, ["name", "flag", "role_type"], "position")

# Step 3: Impute missing flag values
flag_lookup = create_lookup(team_members_df, ["name", "position", "role_type"], "flag")
impute_missing_values(team_members_df, flag_lookup, ["name", "position", "role_type"], "flag")

# Step 4: Impute remaining values based on name and role_type alone
position_lookup_name = create_lookup(team_members_df, ["name", "role_type"], "position")
impute_missing_values(team_members_df, position_lookup_name, ["name", "role_type"], "position")

flag_lookup_name = create_lookup(team_members_df, ["name", "role_type"], "flag")
impute_missing_values(team_members_df, flag_lookup_name, ["name", "role_type"], "flag")

# Step 6: Create a unique ID for combinations of NAME, FLAG, POSITION, and ROLE_TYPE
combination_to_id = {}
current_id = 1

def get_combination_id(row):
    global current_id
    job = 0 if row["position"] == 0 else 1
    key = (
        row["name"],
        row["flag"] if pd.notna(row["flag"]) else None,
        job,
        row["role_type"]
    )
    if key not in combination_to_id:
        combination_to_id[key] = current_id
        current_id += 1
    return combination_to_id[key]

team_members_df["unique_id"] = team_members_df.apply(get_combination_id, axis=1)

# Display the updated DataFrame info
team_members_df.info()




<class 'pandas.core.frame.DataFrame'>
Index: 56567 entries, 0 to 200453
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   team           56567 non-null  object
 1   tournament_id  56567 non-null  int64 
 2   team_id        56567 non-null  int64 
 3   name           56567 non-null  object
 4   position       46842 non-null  object
 5   flag           31734 non-null  object
 6   role_type      56567 non-null  object
 7   clean_team     56567 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.9+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 56567 entries, 0 to 200453
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   team           56567 non-null  object
 1   tournament_id  56567 non-null  int64 
 2   team_id        56567 non-null  int64 
 3   name           56567 non-null  object
 4   position       50036 non-null  object
 5   flag  

In [ ]:
# Configuration parameters
config = {
    "base_k": 200,  # Base K-factor for ELO calculations
    "initial_elo": 1500.0,  # Initial ELO rating for all players and coaches
    "tier_multiplier_factor_pre_2024": 12,  # Pre-2024 tier multiplier factor
    "tier_exponent_pre_2024": 2,  # Pre-2024 tier exponent
    "tier_multiplier_factor_post_2024": 6,  # Post-2024 tier multiplier factor
    "tier_exponent_post_2024": 1,  # Post-2024 tier exponent
}

def calculate_elo_change_common(player_avg_elo, opponent_avg_elo, outcome, base_k, tier_multiplier):
    expected_score = 1 / (1 + 10 ** ((opponent_avg_elo - player_avg_elo) / 400))
    change = (k * tier_multiplier) * (outcome - expected_score)
    return change, expected_score

def update_elo_ratings(matches_df, members_df, tournament_info_df, config):
    base_k = config["base_k"]
    initial_elo = config["initial_elo"]
    tier_multiplier_factor_pre_2024 = config["tier_multiplier_factor_pre_2024"]
    tier_exponent_pre_2024 = config["tier_exponent_pre_2024"]
    tier_multiplier_factor_post_2024 = config["tier_multiplier_factor_post_2024"]
    tier_exponent_post_2024 = config["tier_exponent_post_2024"]

    # Merge and sort data outside the loop
    matches_df = matches_df.dropna(subset=['team1_id', 'team2_id']).sort_values(by='date')

    # Initialize ELO ratings
    members_df['elo_before'] = initial_elo
    members_df['elo_after'] = initial_elo
    members_df['c_elo_before'] = initial_elo
    members_df['c_elo_after'] = initial_elo
    elo_ratings = {}
    coach_elo_ratings = {}

    # Drop columns if they already exist
    matches_df = matches_df.drop(columns=['team1_avg_elo', 'team2_avg_elo', 'team1_expected_outcome', 'team2_expected_outcome'], errors='ignore')

    # Initial values for tier multipliers
    tier_multiplier_factor = tier_multiplier_factor_pre_2024
    tier_exponent = tier_exponent_pre_2024

    for _, match in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Processing matches"):
        # Pre-fetch match values
        match_id = match["match_card_id"]
        date = match["date"]
        winner = match["winner"]
        team1_id = match["team1_id"]
        team2_id = match["team2_id"]
        tier = match["liquipediatier"]

        # Ensure both team1_id and team2_id are present
        if not team1_id or not team2_id:
            continue
        
        # Get team player IDs once and reuse
        team1_players = members_df[(members_df["team_id"] == team1_id) & (members_df["role_type"] == 'player')]["unique_id"].tolist()
        team2_players = members_df[(members_df["team_id"] == team2_id) & (members_df["role_type"] == 'player')]["unique_id"].tolist()
        team1_all_members = members_df[(members_df["team_id"] == team1_id) & (members_df["role_type"].isin(['player', 'staff']))]["unique_id"].tolist()
        team2_all_members = members_df[(members_df["team_id"] == team2_id) & (members_df["role_type"].isin(['player', 'staff']))]["unique_id"].tolist()

        # Skip matches where either team has no players
        if not team1_players or not team2_players:
            continue

        # Update tier multipliers if the date threshold is crossed
        if date >= "2024-01-01" and (tier_multiplier_factor != tier_multiplier_factor_post_2024 or tier_exponent != tier_exponent_post_2024):
            tier_multiplier_factor = tier_multiplier_factor_post_2024
            tier_exponent = tier_exponent_post_2024
        
        if winner not in [1, 2]:
            continue
        
        tier_multiplier = tier_multiplier_factor / ((tier + 1) ** tier_exponent)
        print(f"Multiplier {tier_multiplier}. Tier {tier}. Exponent {tier_exponent}")

        # Retrieve and set ELOs for team members before the match
        for uid in team1_players + team2_players:
            if uid not in elo_ratings:
                elo_ratings[uid] = initial_elo
        
        for uid in team1_all_members + team2_all_members:
            if uid not in coach_elo_ratings:
                coach_elo_ratings[uid] = initial_elo

        team1_avg_elo = sum(elo_ratings[uid] for uid in team1_players) / len(team1_players)
        team2_avg_elo = sum(elo_ratings[uid] for uid in team2_players) / len(team2_players)
        team1_avg_coach_elo = sum(coach_elo_ratings[uid] for uid in team1_all_members) / len(team1_all_members)
        team2_avg_coach_elo = sum(coach_elo_ratings[uid] for uid in team2_all_members) / len(team2_all_members)

        outcome1 = 1 if winner == 1 else 0
        outcome2 = 1 - outcome1

        team1_common_change, team1_expected_outcome = calculate_elo_change_common(team1_avg_elo, team2_avg_elo, outcome1, base_k, tier_multiplier)
        team2_common_change = -team1_common_change  # The change for team 2 is the inverse of team 1

        team1_coach_change, _ = calculate_elo_change_common(team1_avg_coach_elo, team2_avg_coach_elo, outcome1, base_k, tier_multiplier)
        team2_coach_change = -team1_coach_change  # The change for team 2 is the inverse of team 1

        # Store the average ELOs and expected outcomes in matches_df
        matches_df.loc[matches_df['match_card_id'] == match_id, 'team1_avg_elo'] = team1_avg_elo
        matches_df.loc[matches_df['match_card_id'] == match_id, 'team2_avg_elo'] = team2_avg_elo
        matches_df.loc[matches_df['match_card_id'] == match_id, 'team1_expected_outcome'] = team1_expected_outcome
        matches_df.loc[matches_df['match_card_id'] == match_id, 'team2_expected_outcome'] = 1 - team1_expected_outcome

        # Update ELOs for players and coaches
        for team, team_common_change, team_coach_change, team_players, team_all_members in [
            (team1_id, team1_common_change, team1_coach_change, team1_players, team1_all_members),
            (team2_id, team2_common_change, team2_coach_change, team2_players, team2_all_members)
        ]:
            for unique_id in team_players:
                player_elo_before = elo_ratings[unique_id]
                player_elo_after = player_elo_before + team_common_change
                elo_ratings[unique_id] = player_elo_after

                members_df.loc[(members_df['unique_id'] == unique_id) & (members_df['team_id'] == team), 'elo_before'] = player_elo_before
                members_df.loc[(members_df['unique_id'] == unique_id) & (members_df['team_id'] == team), 'elo_after'] = player_elo_after

            for unique_id in team_all_members:
                coach_elo_before = coach_elo_ratings[unique_id]
                coach_elo_after = coach_elo_before + team_coach_change
                coach_elo_ratings[unique_id] = coach_elo_after

                members_df.loc[(members_df['unique_id'] == unique_id) & (members_df['team_id'] == team), 'c_elo_before'] = coach_elo_before
                members_df.loc[(members_df['unique_id'] == unique_id) & (members_df['team_id'] == team), 'c_elo_after'] = coach_elo_after

# Usage example with match_cards_df, team_members_df, and tournament_info_df already loaded
update_elo_ratings(match_cards_df, team_members_df, tournament_info_df, config)

Processing matches:   0%|          | 4/12699 [00:00<06:17, 33.66it/s]

Multiplier 0.48. Tier 4.0. Exponent 2
Multiplier 0.48. Tier 4.0. Exponent 2
Multiplier 0.48. Tier 4.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2


Processing matches:   0%|          | 13/12699 [00:00<05:37, 37.55it/s]

Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2


Processing matches:   0%|          | 21/12699 [00:00<05:29, 38.43it/s]

...

Processing matches:  39%|███▉      | 4988/12699 [02:30<03:42, 34.67it/s]

Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2
Multiplier 0.75. Tier 3.0. Exponent 2


Processing matches:  39%|███▉      | 4993/12699 [02:30<03:52, 33.15it/s]


Multiplier 1.3333333333333333. Tier 2.0. Exponent 2


KeyboardInterrupt: 